<a href="https://colab.research.google.com/github/aabitokh/home_repo/blob/develop/ntb_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd 
import numpy as np 

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

import torch
import transformers
from tqdm import notebook
from torch import tensor


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import re

In [ ]:
DATA_PATH = 'toxic_comments.csv'
#DATA_PATH = '/datasets/toxic_comments.csv'

In [ ]:
def clear_text(text):
    return ' '.join(re.sub(r'[^a-zA-Z\' ]', ' ', text).split())

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
!nvidia-smi

Thu Mar 16 20:35:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    27W /  70W |  11349MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# first look 

In [ ]:
df = pd.read_csv(DATA_PATH)

In [ ]:
df

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [ ]:
df.sample(10)

,Unnamed: 0,text,toxic
15399,15415,'s behaviour \n\n There is currently a discuss...,0
87274,87355,== Richard Huggett == \n\nHello thank you for ...,0
100310,100407,"TFD \n\nRemove all the templates you want, but...",0
113727,113825,REDIRECT Talk:S.L. Benfica (rugby union),0
9546,9559,About Trick Daddy \n\nYou say that the trivia ...,0
32709,32749,"""\n\n The Working Barnstar For your tireless...",0
79383,79459,Phil Donahue\n\nI know he was at WDTN during h...,0
34500,34540,"""\n Unblocking admin should see, the earlier b...",0
105167,105264,"""Unfortunately, Carnildo is not civil himself....",0
113536,113634,Calling someone a fruit is not a personal atta...,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


# data preprocessing

In [ ]:
df = df.drop(['Unnamed: 0'], axis = 1) 

In [ ]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


In [ ]:
df.toxic.value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [ ]:
df['text'] = df['text'].apply(lambda x: clear_text(x))

# BERT

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

#model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = df['text'].apply(lambda x: \
                             tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
padded = torch.LongTensor(padded).to(DEVICE)
attention_mask = torch.LongTensor(attention_mask).to(DEVICE)

torch.Size([159292, 500])

In [ ]:
!nvidia-smi

Thu Mar 16 20:40:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    27W /  70W |  11349MiB / 15360MiB |     39%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model.to(DEVICE)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [154]:
batch_size = 100
embeddings = []


for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = padded[batch_size*i:batch_size*(i+1)]
    attention_mask_batch = attention_mask[batch_size*i:batch_size*(i+1)]

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/1592 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
features = np.concatenate(embeddings)
features

array([[ 0.17851065, -0.0731292 , -0.0242072 , ...,  0.09886129,
         0.3328736 ,  0.36018914],
       [-0.04720632, -0.13178937,  0.34290305, ..., -0.10028522,
         0.53204   ,  0.34827146],
       [ 0.11508776,  0.124497  , -0.09554707, ..., -0.00473316,
         0.6559014 ,  0.28874958],
       ...,
       [ 0.12113795, -0.06216209,  0.05272026, ..., -0.0370937 ,
         0.43502307,  0.3403112 ],
       [ 0.18989868,  0.11428059, -0.13735946, ..., -0.24387924,
         0.34242618,  0.65206164],
       [ 0.09611128,  0.06238674,  0.02186769, ..., -0.04942761,
         0.29244816,  0.2532862 ]], dtype=float32)

In [ ]:
len(features)

159200

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

lr.fit(features, df['toxic'])

ValueError: ignored

In [ ]:
pred = lr.predict(features)

NotFittedError: ignored

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(pred, df['toxic'])

0.98